### Installing necessary libraries

In [1]:
# isntall necessary libraries
! pip install datasets transformers seqeval evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=30f958a9beb910f3713ce324d4e92709095205098796015a22972e8799a0f0b2
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
to

In [2]:
import json
import datasets
import transformers
import seqeval
import torch
from datasets import DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments,Trainer
from seqeval.metrics import classification_report
import numpy as np
from evaluate import load
from collections import defaultdict
from datasets import Dataset
import nltk
from nltk.tokenize import word_tokenize
from datasets import Dataset

nltk.download('punkt')
nltk.download('punkt_tab')

2025-04-21 14:35:59.380173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745246159.623863      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745246159.698727      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Loading the Data Files

In [3]:
# Function to load JSONL file
def load_jsonl(file_path):
    with open(file_path, 'r') as f:
        data = [json.loads(line) for line in f]

    # Print a sample entry for debugging
    print("\n FIRST ENTRY FOR PII DATASET")
    print(json.dumps(data[7], indent=4))  # Pretty-print first entry
    return data

# Load data
data = load_jsonl("/kaggle/input/pii-audios-annotation/PII_audios_annotation.jsonl")


 FIRST ENTRY FOR PII DATASET
{
    "id": 8,
    "text": " hello, my name is lucas martinez. i'm reaching out regarding my recent tax filing. i received a notice about a discrepancy with my bank account number, which is 688388829776. please send any correspondence to 17101 east colfax avenue, aurora, colorado 80011. i would appreciate your assistance in resolving this matter.  additionally, i would like to request a detailed explanation of the discrepancy to better understand the issue.",
    "label": [
        [
            19,
            33,
            "NAME"
        ],
        [
            162,
            174,
            "BANK-ACCOUNT-NO"
        ],
        [
            210,
            258,
            "ADDRESS"
        ]
    ],
    "Comments": []
}


### BIO-Format Mapping 

In [4]:
# Define label mapping for NER
label_map = {
    "NAME": 1, "ADDRESS": 2, "PHONE-NO": 3,
    "BANK-ACCOUNT-NO": 4, "BANK-ROUTING-NO": 5,
    "CREDIT-CARD-NO": 6, "SSN": 7
}

# Reverse mapping for BIO labels
bio_map = {v: k for k, v in label_map.items()}

# Function to convert numerical labels to BIO format
def convert_to_bio(tokens, ner_tags):
    bio_labels = []
    prev_label = 0  # Track previous label

    for i, (token, tag) in enumerate(zip(tokens, ner_tags)):
        if tag == 0:
            bio_labels.append("O")  # Default "Outside" label
        else:
            entity = bio_map[tag]
            if i == 0 or tag != prev_label:  # Start of a new entity
                bio_labels.append(f"B-{entity}")
            else:
                bio_labels.append(f"I-{entity}")
        prev_label = tag  # Update previous label

    return bio_labels

In [5]:
def create_bio_label_map(label_map):
    """
    Generates a mapping of BIO labels to integer IDs.

    Args:
        label_map (dict): Dictionary mapping entity types to IDs.

    Returns:
        dict: BIO label mapping { "O": 0, "B-ENTITY": X, "I-ENTITY": Y, ... }
    """
    bio_map = {"O": 0}  # Default "O" tag

    for entity, idx in label_map.items():
        bio_map[f"B-{entity}"] = idx * 2 - 1  # Example: "B-NAME" → 1
        bio_map[f"I-{entity}"] = idx * 2      # Example: "I-NAME" → 2

    return bio_map

# Create the BIO mapping
bio_label_map = create_bio_label_map(label_map)

# Debugging: Print the generated mapping
print("\nBIO Label Mapping:")
print(bio_label_map)


BIO Label Mapping:
{'O': 0, 'B-NAME': 1, 'I-NAME': 2, 'B-ADDRESS': 3, 'I-ADDRESS': 4, 'B-PHONE-NO': 5, 'I-PHONE-NO': 6, 'B-BANK-ACCOUNT-NO': 7, 'I-BANK-ACCOUNT-NO': 8, 'B-BANK-ROUTING-NO': 9, 'I-BANK-ROUTING-NO': 10, 'B-CREDIT-CARD-NO': 11, 'I-CREDIT-CARD-NO': 12, 'B-SSN': 13, 'I-SSN': 14}


### Data Preparation

In [6]:
label_map = {
    "NAME": 1, "ADDRESS": 2, "PHONE-NO": 3,
    "BANK-ACCOUNT-NO": 4, "BANK-ROUTING-NO": 5,
    "CREDIT-CARD-NO": 6, "SSN": 7
}

def convert_to_hf_format(data):
    examples = {"tokens": [], "ner_tags": []}

    for idx, item in enumerate(data):
        text = item["text"]
        tokens = word_tokenize(text)  # Tokenize using nltk
        labels = [0] * len(tokens)  # Default label (0)

        # Track character positions of each token
        token_spans = []
        char_start = 0
        for token in tokens:
            start_index = text.find(token, char_start)
            end_index = start_index + len(token)
            token_spans.append((start_index, end_index))
            char_start = end_index  # Move forward

        # Assign labels
        for start, end, label in item["label"]:
            for i, (token_start, token_end) in enumerate(token_spans):
                # If token is inside the entity range, label it
                if (token_start >= start and token_end <= end) or (token_start < end and token_end > start):
                    labels[i] = label_map.get(label, 0)

        # Convert to BIO format
        bio_labels = convert_to_bio(text, labels)

        examples["tokens"].append(tokens)
        examples["ner_tags"].append(bio_labels)

        # Debugging: Print first sample
        if idx == 9:
            print("\nSample Tokenized Text:", tokens)
            print("\nSample Token-Label Pair:")
            for token, tag in zip(tokens, bio_labels):
                print(f"{token}: {tag}")

    return Dataset.from_dict(examples)



# Convert and debug dataset
dataset = convert_to_hf_format(data)


Sample Tokenized Text: ['hello', ',', 'my', 'name', 'is', 'sophia', 'thompson', '.', 'i', "'m", 'contacting', 'you', 'about', 'my', 'recent', 'tax', 'return', '.', 'i', 'received', 'a', 'letter', 'indicating', 'a', 'problem', 'with', 'my', 'social', 'security', 'number', 'which', 'is', '778353369.', 'please', 'reach', 'me', 'at', '9565835742', 'or', 'mail', 'me', 'at', '337-dario-fallsuite-518-hartford', ',', 'ct', '06890.', 'i', 'would', 'greatly', 'appreciate', 'your', 'assistance', 'in', 'clarifying', 'this', 'matter', '.', 'you']

Sample Token-Label Pair:
hello: O
,: O
my: O
name: O
is: O
sophia: B-NAME
thompson: I-NAME
.: O
i: O
'm: O
contacting: O
you: O
about: O
my: O
recent: O
tax: O
return: O
.: O
i: O
received: O
a: O
letter: O
indicating: O
a: O
problem: O
with: O
my: O
social: O
security: O
number: O
which: O
is: O
778353369.: B-SSN
please: O
reach: O
me: O
at: O
9565835742: B-PHONE-NO
or: O
mail: O
me: O
at: O
337-dario-fallsuite-518-hartford: B-ADDRESS
,: I-ADDRESS
ct: I

### loading the Model and the Tokenizer

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Define model checkpoint
model_checkpoint = "microsoft/mdeberta-v3-base"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Define the number of labels
num_labels = len(bio_label_map)  # Total number of NER labels

# Load model with the correct number of labels
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=num_labels,id2label={i: label for label, i in bio_label_map.items()},
    label2id=bio_label_map)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Aligning the Token with their Output

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx and word_idx < len(label):
                label_ids.append(bio_label_map.get(label[word_idx], -100))
            else:
                label_ids.append(bio_label_map.get(label[word_idx], -100) if label_all_tokens and word_idx < len(label) else -100)

            previous_word_idx = word_idx

        # Only print debug info for the first 3 examples
        if i < 3:
            print(f"Processing example {i}")
            print(f"Tokens: {examples['tokens'][i]}")
            print(f"Tokenized input IDs: {tokenized_inputs['input_ids'][i]}")
            print(f"Word IDs: {word_ids}")
            print(f"Original Labels: {label}")
            print(f"Aligned Labels: {label_ids}")
            print(f"Length of input IDs: {len(tokenized_inputs['input_ids'][i])}, Length of Labels: {len(label_ids)}\n")

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/1087 [00:00<?, ? examples/s]

Processing example 0
Tokens: ['hello', ',', 'my', 'name', 'is', 'benjamin', 'carter', '.', 'i', "'m", 'contacting', 'you', 'about', 'an', 'issue', 'with', 'my', 'tax', 'return', 'from', 'last', 'year', '.', 'there', 'seems', 'to', 'be', 'a', 'problem', 'with', 'my', 'bank', 'account', 'number', 'for', '873153717', ',', 'and', 'i', 'believe', 'my', 'social', 'security', 'number', '589-90-4308', 'is', 'incorrect', 'in', 'your', 'records', '.', 'i', "'ve", 'already', 'attempted', 'to', 'resolve', 'this', 'issue', 'online', ',', 'but', 'did', "n't", 'receive', 'a', 'response', '.', 'additionally', ',', 'this', 'delay', 'has', 'caused', 'me', 'to', 'miss', 'the', 'filing', 'deadline', ',', 'which', 'could', 'result', 'in', 'penalties', '.', 'please', 'verify', 'the', 'information', 'and', 'reach', 'out', 'to', 'me', 'at', '416-557-3342.', 'thank', 'you', 'for', 'your', 'help', 'in', 'resolving', 'this', 'matter', 'quickly', '.']
Tokenized input IDs: [1, 124394, 260, 262, 1038, 6536, 340, 26

### Train Test Split

In [10]:
from datasets import DatasetDict

# Split dataset (80% train, 10% validation, 10% test)
dataset_dict = tokenized_dataset.train_test_split(test_size=0.2)
train_valid = dataset_dict["train"].train_test_split(test_size=0.1)

# Final splits
final_dataset = DatasetDict({
    "train": train_valid["train"],
    "validation": train_valid["test"],
    "test": dataset_dict["test"]
})

print(final_dataset)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 782
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 87
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 218
    })
})


In [11]:
from collections import Counter

labels = [tag for sequence in dataset["ner_tags"] for tag in sequence]
print("Label Distribution:", Counter(labels))

Label Distribution: Counter({'O': 49748, 'I-ADDRESS': 5854, 'I-NAME': 1391, 'B-NAME': 1071, 'B-PHONE-NO': 783, 'B-ADDRESS': 738, 'B-BANK-ACCOUNT-NO': 525, 'B-BANK-ROUTING-NO': 361, 'B-CREDIT-CARD-NO': 358, 'B-SSN': 347, 'I-PHONE-NO': 67, 'I-CREDIT-CARD-NO': 62, 'I-BANK-ACCOUNT-NO': 36, 'I-SSN': 16, 'I-BANK-ROUTING-NO': 11})


In [12]:
from transformers import DataCollatorForTokenClassification
import torch


# Data collator for padding during batching
data_collator = DataCollatorForTokenClassification(tokenizer)

In [13]:
# Reverse mapping from ID to label
id2label = {idx: label for label, idx in bio_label_map.items()}
# Load the seqeval metric
metric = load("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)  # Convert logits to predicted labels

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

In [14]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Log at the end of each epoch
    logging_strategy="steps",  # Log training loss at intervals
    logging_steps=10,  # Log every 10 steps (adjust as needed)
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    save_total_limit=2,           # Keep only best checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    weight_decay=0.01,
    report_to="none",  # Use "wandb" if logging to Weights & Biases
)
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)



trainer.train()

/tmp/ipykernel_31/1405446203.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.367200,0.198675,0.780591,0.811404,0.795699,0.953358
2,0.114900,0.101139,0.916118,0.977193,0.945671,0.979737
3,0.079500,0.088943,0.926385,0.982456,0.953597,0.981267
4,0.070800,0.087971,0.924217,0.984211,0.953271,0.980630
5,0.062600,0.085849,0.923520,0.985088,0.953311,0.980630


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=245, training_loss=0.2505295866606187, metrics={'train_runtime': 187.55, 'train_samples_per_second': 20.848, 'train_steps_per_second': 1.306, 'total_flos': 282427614338580.0, 'train_loss': 0.2505295866606187, 'epoch': 5.0})

In [15]:
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.08894266188144684,
 'eval_precision': 0.9263854425144747,
 'eval_recall': 0.9824561403508771,
 'eval_f1': 0.953597275436356,
 'eval_accuracy': 0.9812667261373773,
 'eval_runtime': 1.3409,
 'eval_samples_per_second': 64.884,
 'eval_steps_per_second': 4.475,
 'epoch': 5.0}

In [16]:
predictions, labels, _ = trainer.predict(final_dataset["validation"])
predictions = np.argmax(predictions, axis=2)

label_list = [label for label, idx in sorted(bio_label_map.items(), key=lambda x: x[1])]

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'ADDRESS': {'precision': 0.9113924050632911,
  'recall': 0.935064935064935,
  'f1': 0.923076923076923,
  'number': 77},
 'BANK-ACCOUNT-NO': {'precision': 0.907608695652174,
  'recall': 0.9881656804733728,
  'f1': 0.9461756373937678,
  'number': 169},
 'BANK-ROUTING-NO': {'precision': 0.8547008547008547,
  'recall': 0.9615384615384616,
  'f1': 0.9049773755656108,
  'number': 104},
 'CREDIT-CARD-NO': {'precision': 0.9537815126050421,
  'recall': 0.9956140350877193,
  'f1': 0.9742489270386266,
  'number': 228},
 'NAME': {'precision': 0.9754601226993865,
  'recall': 0.9695121951219512,
  'f1': 0.9724770642201835,
  'number': 164},
 'PHONE-NO': {'precision': 0.9384615384615385,
  'recall': 0.9902597402597403,
  'f1': 0.9636650868878356,
  'number': 308},
 'SSN': {'precision': 0.8737864077669902,
  'recall': 1.0,
  'f1': 0.932642487046632,
  'number': 90},
 'overall_precision': 0.9263854425144747,
 'overall_recall': 0.9824561403508771,
 'overall_f1': 0.953597275436356,
 'overall_accuracy': 

In [17]:
from seqeval.metrics import classification_report

print("\nSeqEval Classification Report:")
print(classification_report(true_labels, true_predictions, zero_division=0))



SeqEval Classification Report:
                 precision    recall  f1-score   support

        ADDRESS       0.91      0.94      0.92        77
BANK-ACCOUNT-NO       0.91      0.99      0.95       169
BANK-ROUTING-NO       0.85      0.96      0.90       104
 CREDIT-CARD-NO       0.95      1.00      0.97       228
           NAME       0.98      0.97      0.97       164
       PHONE-NO       0.94      0.99      0.96       308
            SSN       0.87      1.00      0.93        90

      micro avg       0.93      0.98      0.95      1140
      macro avg       0.92      0.98      0.95      1140
   weighted avg       0.93      0.98      0.95      1140



In [21]:
from collections import Counter

# Flatten the label sequences to count the tags
flat_predictions = [label for seq in true_predictions for label in seq]
flat_labels = [label for seq in true_labels for label in seq]

print("\nPredicted Label Distribution:")
print(Counter(flat_predictions))

print("\nTrue Label Distribution:")
print(Counter(flat_labels))


Predicted Label Distribution:
Counter({'O': 36183, 'I-ADDRESS': 2352, 'B-PHONE-NO': 719, 'I-NAME': 522, 'B-BANK-ACCOUNT-NO': 477, 'B-CREDIT-CARD-NO': 471, 'B-NAME': 412, 'B-BANK-ROUTING-NO': 286, 'B-SSN': 243, 'B-ADDRESS': 191})

True Label Distribution:
Counter({'O': 36625, 'I-ADDRESS': 2004, 'B-PHONE-NO': 691, 'I-NAME': 501, 'B-CREDIT-CARD-NO': 434, 'B-BANK-ACCOUNT-NO': 432, 'B-NAME': 406, 'B-BANK-ROUTING-NO': 272, 'B-SSN': 229, 'B-ADDRESS': 195, 'I-CREDIT-CARD-NO': 31, 'I-BANK-ACCOUNT-NO': 15, 'I-PHONE-NO': 11, 'I-SSN': 8, 'I-BANK-ROUTING-NO': 2})


In [22]:
## Save model
model.save_pretrained("ner_model")

In [23]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spm.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [24]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [25]:
config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))

In [26]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

In [27]:
def merge_tokens(ner_results):
    entities = {}
    for entity in ner_results:
        entity_type = entity["entity_group"]
        entity_value = entity["word"].replace("##", "")  # Remove subword prefixes

        # Handle token merging
        if entity_type not in entities:
            entities[entity_type] = []
        if entities[entity_type] and not entity_value.startswith(" "):
            # If the previous token exists and this one isn't a new word, merge it
            entities[entity_type][-1] += entity_value
        else:
            entities[entity_type].append(entity_value)

    return entities

In [28]:
from transformers import pipeline

# Load your fine-tuned NER model
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer, aggregation_strategy="simple")

# List of text examples
examples = [
    "Hi, I’m Mia Thompson. I recently noticed that my electricity bill hasn’t been updated despite making the payment last week. I used account number 4893172051 linked with routing number 192847561. My service was nearly suspended, and I’d appreciate it if you could verify the payment. You can reach me at 727-814-3902 if more information is needed.",

    "Hello, my name is Nathaniel Clark. I need help disputing a charge made to my credit card 4539-2018-4729-5601 for a service I didn’t subscribe to. I’ve already contacted customer support with no resolution. Please check the last three transactions and get back to me at 408-653-1092 with an update.",

    "Hi, this is Avery Scott. I’m concerned about an unauthorized attempt to access my online banking using account number 6049281732. I was notified that someone tried to make a transaction using routing number 281764930. Can you put a freeze on my account until this is resolved? My contact number is 936-284-6075.",

    "Hi, I’m Lily Parker. I submitted my SSN 507-61-2843 last week for verification, but I was told it doesn’t match the one on record. This is delaying my tax refund, and I’m getting concerned. Please double-check my records and let me know if any further documents are needed. You can call me at 516-438-9021.",

    "Hello, I’m Julian Rivera. I’m writing about a phone service bill that I paid using my credit card 6011-5738-4956-2743, but it looks like it was charged twice. My billing address is 892 Willow Bend Lane, Orlando, Florida 32801. I’d appreciate a refund of the duplicate charge. Please call me at 407-726-8193 with an update.",

    "Hi, this is Claire Bennett. I received a disconnect notice even though I paid last month’s water bill using bank account 7021849537 and routing number 310894726. I want to make sure the payment was received on time. If you need anything from my side, you can contact me at 318-764-9205.",

    "Hello, I’m Marcus Foster. I just got a letter stating that my Social Security Number 395-28-6712 doesn’t match your records, which could delay my insurance approval. I’ve double-checked and confirmed my number is correct. Please look into it and call me at 714-805-3126 when resolved.",

    "Hi, I’m Ella Simmons. I noticed a charge for $79.99 on my credit card 5372-8490-1823-9704 for a service I never signed up for. I’d like this transaction investigated and removed. You can reach me at 630-415-8794 to confirm details if needed. Thanks in advance for your support.",

    "This is Logan Hughes. I paid my internet bill using bank account 8294710538, routed through 283716405, but the system still shows a pending balance. Could you confirm the payment was applied correctly? Feel free to contact me at 678-902-1745 if more information is needed.",

    "Hello, I’m Sophia Greene. I’m writing because my utility bill wasn’t processed even after I submitted my payment. I used credit card number 4929-1837-5049-1036, and the payment was shown as 'failed' in your portal. Please confirm the status and call me at 914-875-2093.",

    "Hi, my name is Jacob Morris. I noticed an incorrect debit on my checking account 5910283745, and I’ve already submitted a complaint, but haven’t heard back. It was routed via 209387416. Please investigate and let me know at 715-829-6142.",

    "Hi, this is Madison Lee. I’m applying for a housing program and was informed my SSN 615-94-8701 doesn’t verify. I submitted the documents over a week ago. Could someone please double-check and confirm this? I can be reached at 843-510-2974.",

    "Hello, I’m Owen Carter. I recently moved to 732 Cypress Hill Rd, Raleigh, North Carolina 27603 and updated my billing info. However, I received a bill at my old address again. Could you confirm my current address is on file and contact me at 984-321-7806?",

    "Hi, this is Stella Collins. I got charged twice for a subscription renewal I canceled last month. The charges appeared on my credit card 6011-4739-2893-6501. Can someone check the billing logs and process a refund? I’m available at 417-538-2174.",

    "Hello, I’m Carter Mitchell. There seems to be a mismatch in my employment documents, and HR says my SSN 498-75-1306 isn’t valid in their system. I submitted everything correctly. Please review this error and call me at 775-684-2957 for confirmation.",

    "Hi, I’m Grace Reynolds. I recently submitted my SSN 701-62-1489 for insurance verification, but I received a rejection notice saying it doesn’t match. This has delayed my policy approval, and I’d like it resolved quickly. You can reach me at 973-285-1947 for further clarification.",

    "Hello, my name is Ryan Collins. I noticed a duplicate charge on my credit card 4532-1789-3064-5298 after renewing my gym membership. I only authorized one transaction, but two payments were debited. Please investigate and call me at 646-920-3148 with an update.",

    "Hi, this is Zoe Hamilton. I paid my internet bill using account number 8291037465 and routing number 382910745, but I still received a late notice. Kindly confirm the payment was processed and remove any late fee. My contact number is 502-637-2184.",

    "Hey, I’m Blake Morgan. I recently applied for a loan and used my SSN 319-45-7810, but your system marked it as invalid. I’ve double-checked the number and it’s correct. Can someone verify and contact me at 214-789-6543 as soon as possible?",

    "Hello, I’m Ava Grant. I received a suspicious notification about activity on my bank account 5910283642, routed through 210984763. I did not authorize any recent transactions. Please freeze the account immediately and contact me at 470-893-1674.",

    "Hi, this is Eli Brooks. I made a payment using my credit card 6011-3948-2719-5307 for my electricity bill, but the system shows it’s still unpaid. I want to avoid any interruption in service. Please check and call me at 414-658-2901.",

    "Hello, I’m Hailey Simmons. My water bill has not been updated even though I paid it two days ago from account number 7362019458 with routing number 109384756. Please confirm the payment went through. You can call me at 832-490-8173 if needed.",

    "Hi, I’m Carter James. My employment background check failed because your system shows an error with my SSN 625-93-4781. I’ve verified that the number is correct. Could someone fix this issue and call me at 541-782-3910?",

    "Hey, this is Leah Fisher. I was charged for a service I didn’t use on my credit card 5186-9302-4710-3846. I need help disputing this charge and making sure it doesn’t happen again. Contact me at 909-784-1029 with the resolution.",

    "Hi, I’m Isaac Coleman. My recent transfer didn’t go through even though I used account number 8725091347 and routing number 395721840. I received a failed transaction alert. Can you please investigate and call me at 615-720-3184?",

    "Hello, I’m Nora Wallace. I was trying to update my billing address for my internet service, but your system wouldn’t accept my new zip code. My current address is 1590 Sycamore Trail, Boise, Idaho 83705. Please help me resolve this and contact me at 208-912-4735.",

    "Hi, this is Mason Bryant. My mobile bill was paid using credit card 5421-7803-4910-2764, but the charge appears three times in my statement. Please reverse the extra payments and contact me at 312-784-5063 once it’s fixed.",

    "Hello, I’m Chloe Peterson. I submitted my SSN 509-78-2163 during onboarding, but HR reported it as unmatched. I’ve sent supporting documents already. Please follow up and reach me at 785-394-1628 for any questions.",

    "Hi, I’m Wyatt Ellis. I noticed an old billing address was still on file despite updating it last month. It should be 2078 Lakeview Avenue, Charlotte, North Carolina 28210. Kindly update your records and let me know at 980-657-3102.",

    "This is Stella Young. I got a notification about suspicious activity on my account 7382910546 via routing number 189273645. I haven’t made any recent transactions. Please secure my account and call me at 872-561-4309 with a report."
]


# Process each example through the NER model
for i, example in enumerate(examples, 1):
    ner_results = nlp(example)
    cleaned_entities = merge_tokens(ner_results)

    # Display results
    print(f"NER Results for Example {i}:")
    for entity_type, values in cleaned_entities.items():
        print(f"  {entity_type}: {', '.join(values)}")
    print("\n" + "-"*80 + "\n")


Device set to use cuda:0


NER Results for Example 1:
  NAME: Mia Thompson
  BANK-ACCOUNT-NO: 4893172051
  BANK-ROUTING-NO: 192847561
  PHONE-NO: 727-814-3902

--------------------------------------------------------------------------------

NER Results for Example 2:
  NAME: Nathaniel Clark
  CREDIT-CARD-NO: 4539-2018-4729-5601
  PHONE-NO: 408-653-1092

--------------------------------------------------------------------------------

NER Results for Example 3:
  NAME: Avery Scott
  BANK-ACCOUNT-NO: 6049281732.
  BANK-ROUTING-NO: 281764930.
  PHONE-NO: 936-284-6075.

--------------------------------------------------------------------------------

NER Results for Example 4:
  NAME: Lily Parker
  SSN: 507-61-2843
  PHONE-NO: 516-438-9021.

--------------------------------------------------------------------------------

NER Results for Example 5:
  NAME: Julian Rivera
  CREDIT-CARD-NO: 6011-5738-4956-2743
  ADDRESS: 892 Willow Bend Lane, Orlando, Florida 32801.
  PHONE-NO: 407-726-8193

--------------------------

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


NER Results for Example 8:
  NAME: Ella Simmons
  CREDIT-CARD-NO: 5372-8490-1823-9704
  PHONE-NO: 630-415-8794

--------------------------------------------------------------------------------

NER Results for Example 9:
  NAME: Logan Hughes
  BANK-ACCOUNT-NO: 8294710538
  BANK-ROUTING-NO: 283716405
  PHONE-NO: 678-902-1745

--------------------------------------------------------------------------------

NER Results for Example 10:
  NAME: Sophia Greene
  CREDIT-CARD-NO: 4929-1837-5049-1036
  PHONE-NO: 914-875-2093.

--------------------------------------------------------------------------------

NER Results for Example 11:
  NAME: Jacob Morris
  BANK-ACCOUNT-NO: 5910283745
  PHONE-NO: 715-829-6142.

--------------------------------------------------------------------------------

NER Results for Example 12:
  NAME: Madison Lee
  SSN: 615-94-8701
  PHONE-NO: 843-510-2974.

--------------------------------------------------------------------------------

NER Results for Example 13:
  

In [29]:
# Function to redact named entities in a given text
def redact_text(text):
    ner_results = nlp(text)  # Get NER results
    cleaned_entities = merge_tokens(ner_results)  # Merge tokens for multi-token entities

    redacted_text = text
    for entity_type, values in cleaned_entities.items():
        for value in values:
            # Replace each identified entity with "[REDACTED]"
            redacted_text = redacted_text.replace(value, "[REDACTED]")

    return redacted_text

# Redact all texts in the list
redacted_examples = [redact_text(text) for text in examples]

# Print redacted results
for i, redacted_text in enumerate(redacted_examples, 1):
    print(f"Redacted Example {i}:\n{redacted_text}\n")
    print("-" * 80)

Redacted Example 1:
Hi, I’m [REDACTED]. I recently noticed that my electricity bill hasn’t been updated despite making the payment last week. I used account number [REDACTED] linked with routing number [REDACTED]. My service was nearly suspended, and I’d appreciate it if you could verify the payment. You can reach me at [REDACTED] if more information is needed.

--------------------------------------------------------------------------------
Redacted Example 2:
Hello, my name is [REDACTED]. I need help disputing a charge made to my credit card [REDACTED] for a service I didn’t subscribe to. I’ve already contacted customer support with no resolution. Please check the last three transactions and get back to me at [REDACTED] with an update.

--------------------------------------------------------------------------------
Redacted Example 3:
Hi, this is [REDACTED]. I’m concerned about an unauthorized attempt to access my online banking using account number [REDACTED] I was notified that so

In [31]:
from transformers import pipeline

# Load your fine-tuned NER model
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer, aggregation_strategy="simple")

# List of text examples
examples = [
    "Hello, my name is Daniel Brooks. I’m reaching out regarding an issue with my mortgage payment. I recently made a payment using my bank account 3857194621, but I received a notification that the payment was declined due to an account mismatch. I believe there’s an error in how my details were processed. This delay is concerning as it might result in late fees. Please verify my details and reach out to me at 312-674-2251 as soon as possible. Thank you for your assistance.",

    "Hi, this is Olivia Carter. I’ve been experiencing a service disruption with my electricity at 217 Maple Street, San Diego, CA 92103. Despite making my monthly payment on time, my service was unexpectedly disconnected. I tried reaching customer support, but I haven’t been able to get through. This is causing significant inconvenience for my household. Could you please check the status of my payment and restore the service at the earliest?",

    "Hi, I’m Jonathan Reed. I recently noticed an unauthorized withdrawal from my checking account 2609-4753-6712-8953 that I did not authorize. I haven’t lost my card or shared my account details with anyone. I contacted my bank, but they advised me to reach out to you for further investigation. Could you please look into this issue and put a temporary hold on my account to prevent further transactions? Please call me at 845-372-1098 for updates. Thank you for your time.",

    "Hi, this is Laura Smith. I recently received a notification about a discrepancy in my Social Security details linked to my employment records. My Social Security Number 621-87-3251 appears to be mismatched in your system, which could potentially affect my payroll and benefits. I’d appreciate it if someone could verify and correct this issue as soon as possible. Please mail the confirmation to 1295 West Grove Ave, Springfield, IL 62704. Thank you for your help.",

    "Hello, I’m Ethan Parker. I have a billing issue with my internet service. My service address is 5521 Walnut Drive, Austin, TX 78745. I recently paid my bill using my credit card 5123-4568-9875-2145, but it seems the payment wasn’t processed correctly, and I’ve been charged twice for the same billing cycle. Could you please review my account and refund the extra charge? You can contact me at 678-214-8895 for any further information.",

     "hi, i'm william harris. i'm having an issue with my auto insurance. i filed a claim but it hasn't been processed yet. my account number is 43786152 and my driver's license number is 705-70-0962. would you look into this? you can reach me at 541-273-9084.  and my address is 9844 parkview street, portland, oregon 97223. thank you for your help.",

    "hello, this is liam wright. i recently paid my rent, but it hasn't been reflected in my account. my account number is 390-283-7465 and the payment was $1,200. my driver's license number is y9349074. can you verify the payment status? my contact number is 310-539-7465.  7291 and my address is 8317 oak avenue, los angeles, california 9001. thank you for your assistance.",


    "hello, this is jackson smith. i'm contacting you about an issue with my student loan account. my account number is 850-192-3821 and i recently made a payment of $1000 but it hasn't been applied to my balance. can you please investigate? my passport number is b98765432.  please contact me at 818-523-9074 or send mail to 1937 beverly bollywood, los angeles, ca 90057. thank you for your help.",


    "hi, i'm matthew hall. i'm writing regarding a recent charge on my bank account 158-039-6728 for $3.50 which i do not recognize. i believe my account might have been compromised. my address is 1421 oakwood drive, denver, colorado 80239. please investigate this matter and contact me at 303-763-0491. thank you for your attention to this",


    "hi, i am lucas clark. i recently made a payment of $1,500 on my loan, but the payment hasn't been reflected. my bank routing number is 894-73-8591 and my passport number is l12345678. can you check the status? my contact number is 404-438-2573.  and my address is 21349 atlas, georgia 30303. thank you."

]



# Process each example through the NER model
for i, example in enumerate(examples, 1):
    ner_results = nlp(example)
    cleaned_entities = merge_tokens(ner_results)

    # Display results
    print(f"NER Results for Example {i}:")
    for entity_type, values in cleaned_entities.items():
        print(f"  {entity_type}: {', '.join(values)}")
    print("\n" + "-"*80 + "\n")


Device set to use cuda:0


NER Results for Example 1:
  NAME: Daniel Brooks
  BANK-ACCOUNT-NO: 3857194621
  PHONE-NO: 312-674-2251

--------------------------------------------------------------------------------

NER Results for Example 2:
  NAME: Olivia Carter
  ADDRESS: 217 Maple Street, San Diego, CA 92103.

--------------------------------------------------------------------------------

NER Results for Example 3:
  NAME: Jonathan Reed
  BANK-ACCOUNT-NO: 2609-4753-6712-8953
  PHONE-NO: 845-372-1098

--------------------------------------------------------------------------------

NER Results for Example 4:
  NAME: Laura Smith
  SSN: 621-87-3251
  ADDRESS: 1295 West Grove Ave, Springfield, IL 62704.

--------------------------------------------------------------------------------

NER Results for Example 5:
  NAME: Ethan Parker
  ADDRESS: 5521 Walnut Drive, Austin, TX 78745.
  CREDIT-CARD-NO: 5123-4568-9875-2145
  PHONE-NO: 678-214-8895

-----------------------------------------------------------------------

In [32]:
# Function to redact named entities in a given text
def redact_text(text):
    ner_results = nlp(text)  # Get NER results
    cleaned_entities = merge_tokens(ner_results)  # Merge tokens for multi-token entities

    redacted_text = text
    for entity_type, values in cleaned_entities.items():
        for value in values:
            # Replace each identified entity with "[REDACTED]"
            redacted_text = redacted_text.replace(value, "[REDACTED]")

    return redacted_text

# Redact all texts in the list
redacted_examples = [redact_text(text) for text in examples]

# Print redacted results
for i, redacted_text in enumerate(redacted_examples, 1):
    print(f"Redacted Example {i}:\n{redacted_text}\n")
    print("-" * 80)

Redacted Example 1:
Hello, my name is [REDACTED]. I’m reaching out regarding an issue with my mortgage payment. I recently made a payment using my bank account [REDACTED], but I received a notification that the payment was declined due to an account mismatch. I believe there’s an error in how my details were processed. This delay is concerning as it might result in late fees. Please verify my details and reach out to me at [REDACTED] as soon as possible. Thank you for your assistance.

--------------------------------------------------------------------------------
Redacted Example 2:
Hi, this is [REDACTED]. I’ve been experiencing a service disruption with my electricity at [REDACTED] Despite making my monthly payment on time, my service was unexpectedly disconnected. I tried reaching customer support, but I haven’t been able to get through. This is causing significant inconvenience for my household. Could you please check the status of my payment and restore the service at the earlies

In [ ]:
from huggingface_hub import login

login(token="your hugging face token")

In [39]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Load your model and tokenizer from saved directories
model = AutoModelForTokenClassification.from_pretrained("ner_model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

# Push both to the Hugging Face Hub under a new repo name
repo_name = "pii-entity-extractor"  # change this to any name you want on the hub

# Push to your account
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/AI-Enthusiast11/pii-entity-extractor/commit/5bee20c1d19f9655029cc63f2adf05317a2d2c90', commit_message='Upload tokenizer', commit_description='', oid='5bee20c1d19f9655029cc63f2adf05317a2d2c90', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AI-Enthusiast11/pii-entity-extractor', endpoint='https://huggingface.co', repo_type='model', repo_id='AI-Enthusiast11/pii-entity-extractor'), pr_revision=None, pr_num=None)